# Implémentation : Branche `scimba_jax_dg`

---

## 1. Mapping

* `@dataclass(frozen=True)` $\rightarrow$ génère `__init__`, `__str__`, `__eq__` $\rightarrow$ Instance immuable.
* **Mapping** = Composition de `InvertibleFunction` et `InvertibleModule`.

$$f = f_N \circ f_{N-1} \circ ... \circ f_1$$

## 2. Quadrature (Exemple avec Gauss en 2D)



$$ \int_{-1}^1 \int_{-1}^1 f(x,y) dx dy = \sum_{i=1}^n \sum_{j=1}^n w_i w_j f(x_i, y_j) $$

où $n$ est `ordre` = `n_points` .

**Dimensions des tenseurs :**
* `points_1D`,`weights_1D` : $(n,)$, $(n,)$
* `volumic_points`, `volumic_weights` : $(n^d, d)$, $(n^d,)$
* `surfacic_points` : liste de taille $2d$ (2 faces par dim) où chaque élément est $(n^{d-1}, d)$

    `surfacic_weights` : liste de taille $2d$ (2 faces par dim) où chaque élément est $(n^{d-1},)$

## 3. Maillage en 1D (Mesh1D)

Passage sur l'élément de référence pour l'intégration :

$$ \int_{\Omega} f(X) dX = \int_{\Omega_{\text{ref}}} f(\Phi(\xi)) |\det J_{\Phi}(\xi)| d\xi $$

Discrétisation :

$$ \int_{\Omega} f(X) dX  \approx \sum w_i f(X_i) $$

$$ \int_{\Omega_{\text{ref}}} f(\Phi(\xi)) |\det J_{\Phi}(\xi)| d\xi \approx \sum |\det J_{\Phi}(\xi_i)| w_i f(\Phi(\xi_i)) $$

## 4. Bases

On note :

- $d$ : la dimension physique (pour l'instant $d=1$)
- $n_c$ : le nombre de cellules
- $n_b$ : l'ordre de la base (`order`)
- $n_u$ :  le nombre total de variables (`out_dim`)
- $n_{\text{quad}}$ : le nombre de points de quadrature

Chaque variable peut s'écrire sous la forme :

$$ u(x) = \sum_{k=0}^{n_c-1} \sum_{i=0}^{n_b-1} u_{k,i} \varphi_{k,i}(x) $$

### Analytic Basis (Taylor)
$$ \varphi_{k,i}(x) = \frac{(x - x_k)^i}{i!} \quad \longrightarrow \quad (n_b, n_u). $$
où $x_k$ est le centre de la cellule $k$, 

**Dimensions :**
* $x$ : pts de quadrature (global) $\quad \longrightarrow \quad (n_c, n_{\text{quad}}, d)$
* $y = \{\varphi_{k,i}(x)\}$ : évaluation des fonctions de bases au pts de quadrature $\quad \longrightarrow (n_c, n_{\text{quad}}, n_b, n_u)$
* $\partial_x y \quad \longrightarrow (n_c, n_{\text{quad}}, n_b, n_u, d)$

### Bases paramétrées (Machine Learning)
* **Patchwise Parametric Basis :** 
  $$ \varphi_{k,i}^\theta(x) = v_\theta(x) \varphi_{k,i}(x) $$
* **Cellwise Parametric Basis :**
  $$ \varphi_{k,i}^\theta(x) = v_{k,\theta}(x) \varphi_{k,i}(x) $$

## 5. Variables

$$ u_h(x) = \sum_{k=0}^{n_c-1} \sum_{i=0}^{n_b-1} u_{k,i} \varphi_{k,i}(x) \mathbb{1}_{\{x \in C_k\}} $$

### Évaluation locale

$x \in C_k$, avec $k \in [\![0, n_c-1]\!]$.
$$ u_h(x) = \sum_{i=0}^{n_b-1} u_{k,i} \varphi_{k,i}(x) $$

### Projection de $f$ sur l'espace DG
$\hookrightarrow$ Chercher les $u_{k,i}$ tels que $u_h$ soit la fonction approchée de $f$.

Pour $C_k$ :
$$ \int_{C_k} (f - u_h) \varphi_{k,j} = 0 \quad \forall j $$

$$ \underbrace{\int_{C_k} f \cdot \varphi_{k,j}}_{b_j} = \sum_{i=0}^{n_b-1} u_{k,i} \underbrace{\int_{C_k} \varphi_{k,i} \cdot \varphi_{k,j}}_{M_{i,j}} $$

$\hookrightarrow$ **Projecteur local :** $$ M_k U_k = b_k $$

**Dimensions :**

- `x = m.evaluate_mesh_points()` : pts de quadrature (global) $\quad \longrightarrow \quad (n_c, n_{\text{quad}}, d)$
- `variables.evaluate_quad(x)` : évaluation des variables sur les pts de quadrature $\quad \longrightarrow \quad (n_c, n_{\text{quad}}, n_u)$.

Avec Victor : les termes de volume (sur l'équation elliptique, pas d'inversion de matrice)